In [ ]:
from urllib.request import urlretrieve

# Download all required scripts
repo = "Jongseok-han/ww_tools/main/basket_tool/"
base_url = f"https://raw.githubusercontent.com/{repo}"
files = [
    "basket_agents.py",
    "basket_agent_tools.py",
    "basket_pipeline.py"
]

for file in files:
    urlretrieve(base_url + file, file)
    print(f"✅ Downloaded {file}")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import Xano API wrapper functions
from xano_api import (
    get_all_companies_data,
    df_join,
    preprocess_string_numbers)

In [4]:
# Load company data - it already has all filtering fields built-in!
companies = get_all_companies_data('company', as_dataframe=True)
print(f"✅ Loaded {len(companies)} companies")

df, cols = preprocess_string_numbers(companies.copy())

✅ Loaded 1043 companies
📊 Summary: Converted 2 columns to numeric


In [5]:
# Load detailed analysis data
cyclicality = get_all_companies_data('cyclicality', as_dataframe=True)
growth = get_all_companies_data('growth', as_dataframe=True)
valuation = get_all_companies_data('valuation', as_dataframe=True)
lifecycle = get_all_companies_data('lifecycle', as_dataframe=True)
cagr_prediction = get_all_companies_data('cagr_prediction', as_dataframe=True)

df = df_join(df, cyclicality, 'cyclicality')
df = df_join(df, growth, 'growth')
df = df_join(df, valuation, 'valuation')
df = df_join(df, lifecycle, 'lifecycle')
df = df_join(df, cagr_prediction, 'cagr')

print(f"\n✅ Comprehensive df created: {len(df)} companies × {len(df.columns)} columns")
print(f"   All analysis data joined using original 'id' column only")
print(f"\n🔍 Sample data:")
df[['ticker', 'name', 'sector', 'market_cap', 'valuation_classification']].head(3)


✅ Comprehensive df created: 1047 companies × 38 columns
   All analysis data joined using original 'id' column only

🔍 Sample data:


,ticker,name,sector,market_cap,valuation_classification
0,CCL,Carnival Corporation & plc,Consumer Cyclical,36.0,Cyclical Business
1,NFLX,"Netflix, Inc.",Communication Services,510.0,Over Valued
2,ACM,Aecom,Industrials,17.0,Cyclical Business


In [6]:
# Workflow Design

# Step 1: Filter with `df` (companies table)
# - Fast filtering on market_cap, sector, classifications, etc.

# Step 2: Get detailed insights from other tables
# - After filtering, join with `cyclicality`, `growth`, `valuation`, etc.
# - These contain rich `data` columns with reasoning, analysis, recommendations
# - Example: `cyclicality_data` has classification reasoning and investor strategy

In [7]:
# Import the basket pipeline
from basket_pipeline import find_investment_opportunities, display_recommendations

print(f"Dataset: {len(df)} companies")
print(f"Workflow:")
print(" 1. LoopAgent: [QueryGenerator, QueryEvaluator] - Iterative query refinement")
print(" 2. SequentialAgent: [RefinementLoop, RankingAgent] - Filter & rank")
print(f"\nQuick preview:")
df[['ticker', 'name', 'sector', 'market_cap', 'valuation_classification', 'growth_classification']].head(3)

Dataset: 1047 companies
Workflow:
 1. LoopAgent: [QueryGenerator, QueryEvaluator] - Iterative query refinement
 2. SequentialAgent: [RefinementLoop, RankingAgent] - Filter & rank

Quick preview:


,ticker,name,sector,market_cap,valuation_classification,growth_classification
0,CCL,Carnival Corporation & plc,Consumer Cyclical,36.0,Cyclical Business,Moderate Earnings Growth
1,NFLX,"Netflix, Inc.",Communication Services,510.0,Over Valued,Strong Earnings Growth
2,ACM,Aecom,Industrials,17.0,Cyclical Business,Moderate Earnings Growth


In [8]:
# Example 1: Undervalued companies for long-term investment
recommendations = find_investment_opportunities(
    df=df,
    criteria="undervalued companies within non-tech fields for risk-averse long-term investment",
    top_n=5
)

# Display recommendations
display_recommendations(recommendations)

# Access individual recommendations
if recommendations:
    print(f"\n🏆 Top Pick: {recommendations[0].ticker}")
    print(f"   Score: {recommendations[0].score}/100")
    print(f"   Breakdown: {recommendations[0].score_breakdown}")


COMPANY FILTERING PIPELINE
Criteria: undervalued companies within non-tech fields for risk-averse long-term investment
Target: Top 5 companies
Max refinement iterations: 3




   📝 INITIAL (Iteration 1):
   Query: df[(df['valuation_classification'] == 'Under Valued') & (df['sector'] != 'Technology')]


   ✓ Result count: 98
   📊 Sorted by cagr_eps_forecast_10yr (descending) - top 50 selected for ranking
   ✅ APPROVED: 98 results is adequate (target: 5-100)
   🛑 Exiting refinement loop



   📊 Providing 50 companies to agent for ranking
   📋 Available tickers (sample): DKNG, CAVA, SNAP, ABNB, ETSY


   ✅ Stored 5 ranked companies

PIPELINE COMPLETE
Query refinement iterations: 1
Filtered companies: 98
Ranked recommendations: 5


TOP COMPANY RECOMMENDATIONS

#1 - PRI (Primerica, Inc.) - Score: 88/100
----------------------------------------------------------------------------------------------------
Score Breakdown: Strong alignment (38/40) + growth (17/20) + valuation (17/20) + risk (16/20)

Recommendation: Primerica demonstrates a robust business model focused on providing financial services to middle-income families, resulting in stable and predictable revenue with a consistent history, undervalued, and attractive dividend yield.

Investment Thesis: Defensive business model with high returns on equity.

Key Metrics:
  • market_cap: 8.0
  • sector: Financial Services
  • cagr_eps_forecast_10yr: 9.0
  • valuation_classification: Under Valued
  • growth_classification: Moderate Earnings Growth
  • cyclicality_classification: Consistent Compounding Earnings

#2 - CNM (Core & Main, I

In [9]:
# Example 2: Grocery-related companies with specific criteria
recommendations = find_investment_opportunities(
    df=df,
    criteria="grocery-related companies with market cap over 100 billion and EPS growth > 5% per year",
    top_n=5
)

# Display recommendations
display_recommendations(recommendations)

# Access individual recommendations
if recommendations:
    print(f"\n🏆 Top Pick: {recommendations[0].ticker}")
    print(f"   Score: {recommendations[0].score}/100")
    print(f"   Breakdown: {recommendations[0].score_breakdown}")


COMPANY FILTERING PIPELINE
Criteria: grocery-related companies with market cap over 100 billion and EPS growth > 5% per year
Target: Top 5 companies
Max refinement iterations: 3




   📝 INITIAL (Iteration 1):
   Query: df[(df['sector'] == 'Consumer Defensive') & (df['market_cap'] > 100) & (df['cagr_eps_forecast_10yr'] > 5)]


   ✓ Result count: 6
   ✅ APPROVED: 6 results is adequate (target: 5-100)
   🛑 Exiting refinement loop



   📊 Providing 6 companies to agent for ranking
   📋 Available tickers (sample): COST, PM, KO, PEP, WMT


   ✅ Stored 5 ranked companies

PIPELINE COMPLETE
Query refinement iterations: 1
Filtered companies: 6
Ranked recommendations: 5


TOP COMPANY RECOMMENDATIONS

#1 - COST (Costco Wholesale Corporation) - Score: 88/100
----------------------------------------------------------------------------------------------------
Score Breakdown: Strong alignment (38/40) + Growth (18/20) + Valuation (16/20) + Risk/Reward (16/20)

Recommendation: Costco demonstrates exceptional financial strength, consistent earnings growth, and a defensive business model. Although currently overvalued, its operational excellence and membership model justify its status as a top-ranked company.

Investment Thesis: High-quality defensive compounder with a strong membership model driving recurring revenue and earnings growth.

Key Metrics:
  • market_cap: 412.0
  • sector: Consumer Defensive
  • cagr_eps_forecast_10yr: 9.0
  • valuation_classification: Over Valued
  • growth_classification: Moderate Earnings Growth
  • 

In [10]:
# Example 3: Small cap cyclical companies for long-term investment
recommendations = find_investment_opportunities(
    df=df,
    criteria="FIND CYCLICAL COMPANIES WHICH ARE TRADING CLOSE TO THE RECOMMENDED BUY PRICE FOR RISK-AVERSE LONG-TERM INVESTMENT",
    top_n=5
)

# Display recommendations
display_recommendations(recommendations)

# Access individual recommendations
if recommendations:
    print(f"\n🏆 Top Pick: {recommendations[0].ticker}")
    print(f"   Score: {recommendations[0].score}/100")
    print(f"   Breakdown: {recommendations[0].score_breakdown}")


COMPANY FILTERING PIPELINE
Criteria: FIND CYCLICAL COMPANIES WHICH ARE TRADING CLOSE TO THE RECOMMENDED BUY PRICE FOR RISK-AVERSE LONG-TERM INVESTMENT
Target: Top 5 companies
Max refinement iterations: 3




   📝 INITIAL (Iteration 1):
   Query: df[df['cyclicality_classification'] == 'Cyclical Earnings']


   ✓ Result count: 461
   📊 Sorted by cagr_eps_forecast_10yr (descending) - top 50 selected for ranking
   ❌ REJECTED: Too many results: 461 (target: 5-100)
   💡 Refinement guidance: Too many results: 461 (target: 5-100). Tighten ONLY numerical thresholds - DO NOT change categorical filters.



   🔄 REFINEMENT (Iteration 2):
   Query: df[(df['cyclicality_classification'] == 'Cyclical Earnings') & (df['market_cap'] < 200)]
   Previous count: 461


   ✓ Result count: 438
   📊 Sorted by cagr_eps_forecast_10yr (descending) - top 50 selected for ranking
   ❌ REJECTED: Too many results: 438 (target: 5-100)
   💡 Refinement guidance: Too many results: 438 (target: 5-100). Tighten ONLY numerical thresholds - DO NOT change categorical filters.



   🔄 REFINEMENT (Iteration 3):
   Query: df[(df['cyclicality_classification'] == 'Cyclical Earnings') & (df['market_cap'] < 100)]
   Previous count: 438


   ✓ Result count: 410
   📊 Sorted by cagr_eps_forecast_10yr (descending) - top 50 selected for ranking
   ❌ REJECTED: Too many results: 410 (target: 5-100)
   💡 Refinement guidance: Too many results: 410 (target: 5-100). Tighten ONLY numerical thresholds - DO NOT change categorical filters.



   📊 Providing 50 companies to agent for ranking
   📋 Available tickers (sample): ARES, LNG, SMCI, MSTR, Z

PIPELINE COMPLETE
Query refinement iterations: 3
Filtered companies: 410
Ranked recommendations: 0


No recommendations found
